# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using AtomsBuilder
using DFTK
using LinearAlgebra
using PseudoPotentialData

pseudopotentials = PseudoFamily("dojo.nc.sr.pbesol.v0_4_1.standard.upf")
model = model_DFT(bulk(:Si); functionals=PBEsol(), pseudopotentials)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397855702409                   -0.90    5.0   26.9ms
  2   -8.400230649558       -2.62       -1.74    1.0   19.1ms
  3   -8.400407215966       -3.75       -2.99    1.5   20.2ms
  4   -8.400427865079       -4.69       -3.00    3.2   25.3ms
  5   -8.400427942122       -7.11       -3.05    1.0   19.5ms
  6   -8.400428145596       -6.69       -4.68    1.0   19.1ms
  7   -8.400428151779       -8.21       -4.44    2.8   75.9ms
  8   -8.400428152183       -9.39       -5.22    1.0   19.6ms
  9   -8.400428152207      -10.62       -6.36    1.5   20.4ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -8.397858886486                   -0.90           5.2    888ms
  2   -8.400385956291       -2.60       -1.79   0.80    2.2    491ms
  3   -8.400422755765       -4.43       -3.03   0.80    1.0    208ms
  4   -8.400428100440       -5.27       -3.39   0.80    2.5   20.4ms
  5   -8.400428148321       -7.32       -4.40   0.80    1.0   16.5ms
  6   -8.400428152135       -8.42       -5.67   0.80    2.5   20.1ms
  7   -8.400428152209      -10.13       -5.87   0.80    3.2   22.5ms
  8   -8.400428152209      -12.51       -7.27   0.80    1.0   16.8ms


## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

┌ Warning: x_tol is deprecated. Use x_abstol or x_reltol instead. The provided value (-1) will be used as x_abstol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:110
┌ Warning: f_tol is deprecated. Use f_abstol or f_reltol instead. The provided value (-1) will be used as f_reltol.
└ @ Optim ~/.julia/packages/Optim/7krni/src/types.jl:120
n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   +1.406907334485                   -1.10    3.57s
  2   -1.290860587140        0.43       -0.66    142ms
  3   -4.295833361420        0.48       -0.39   43.4ms
  4   -5.796477310654        0.18       -0.48   43.6ms
  5   -7.447438602780        0.22       -0.70   78.9ms
  6   -7.783679954424       -0.47       -1.31   32.4ms
  7   -8.174286616945       -0.41       -1.66   32.0ms
  8   -8.273075317775       -1.01       -1.83   32.4ms
  9   -8.334395551315       -1.21       -2.05   32.2ms
 10   -8.363409229897       -1.54       -2.05   32

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.397835008758                   -0.90    5.0   26.3ms


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -8.400427985742                   -1.79    8.33s
  2   -8.400428152209       -6.78       -4.04    3.24s
  3   -8.400428152209      -14.75       -7.84    128ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 1.242275604599734e-6
|ρ_newton - ρ_scfv| = 9.07347330865052e-8
|ρ_newton - ρ_dm|   = 2.5626092002056975e-6
